## TO DO

1. Dagger save and loading in make vcf functions

2. filter vcf lines to return important bits

3. check reference chromosome

4. pull data from gene and variant databases (prefereably already hosted online)

In [1]:
using VCFTool

┌ Info: Precompiling VCFTool [8f8d80d9-e796-45a3-8ef2-c0461a3110a4]
└ @ Base loading.jl:1278


In [2]:
# For viewing DataFrames

ENV["COLUMNS"]=100

ENV["LINES"]=200

# For viewing JuliaDB tables and arrays

using IndexedTables

IndexedTables.set_show_compact!(false)

false

In [3]:
input_dir = "../input/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_738_1000_file_path = joinpath(input_dir, "738_variants_1000.vcf.gz")

vcf_769_3_file_path = joinpath(input_dir, "769_3_variants.vcf.gz")

vcf_test_file_path = joinpath(input_dir, "test.vcf.gz")

vcf_file_path_to_use = vcf_738_file_path

"../input/738_variants.vcf.gz"

In [ ]:
# timerinfo, tmp = tracktime() do
#          println("yes")
# end

## Search by Variant

In [4]:
vcf = make_vcf_ndsparse(vcf_file_path_to_use)

1-d NDSparse with 5217068 values (10 field named tuples):
1       │ CHROM   POS         ID   REF          ALT                QUAL    FILTER                                INFO                                                    FORMAT                                     GERM
────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
2       │ "chr1"  "10439"     "."  "AC"         "A"                "72"    "PASS"                                "CIGAR=1M1D;RU=C;REFREP=4;IDREP=3;MQ=9"                 "GT:GQ:GQX:DPI:AD:ADF:ADR:FT:PL"           "0/1:31:5:7:2,5:0,1:2,4:PASS:108,0,28"
3       │ "chr1"  "13284"     "."  "G"          "A"                "60"    "LowGQX;NoPassedVariantGTs"           "SNVHPOL=4;MQ=17"                                       "GT:GQ:GQX:DP

In [5]:
vcf_table = make_vcf_indexedtable(vcf_file_path_to_use)

LoadError: InterruptException:
table(::Val{:serial}, ::NamedTuple{(:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),NTuple{10,WeakRefStrings.StringArray{String,1}}}; pkey::Tuple{Int64}, chunks::Nothing, perms::Array{IndexedTables.Perm,1}, presorted::Bool, copy::Bool, cardinality::Array{Missing,1}) at /Users/kate/.julia/packages/IndexedTables/tRvfu/src/indexedtable.jl:101
table(::NamedTuple{(:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),NTuple{10,WeakRefStrings.StringArray{String,1}}}; chunks::Nothing, kwargs::Base.Iterators.Pairs{Symbol,Any,Tuple{Symbol,Symbol},NamedTuple{(:pkey, :copy),Tuple{Tuple{Int64},Bool}}}) at /Users/kate/.julia/packages/IndexedTables/tRvfu/src/indexedtable.jl:137
table(::StructArrays.StructArray{NamedTuple{(:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),NTuple{10,String}},1,NamedTuple{(:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),NTuple{10,WeakRefStrings.StringArray{Str

In [ ]:
import Pkg; Pkg.add("Dagger")

using Dagger

Dagger.save(vcf_table, "../input/test_vcf_table.vcf")

using Dagger

dagger_load_vcf = Dagger.load("../input/test_vcf_ndsparse_save.vcf")

## Search for variant

In [ ]:
using Dates

start_time = now()

filter(i -> (i.CHROM == 1) && (i.POS == 10144), vcf_table)

end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")

## Search for region

In [ ]:
using Dates

start_time = now()

println(filter(i -> (i.CHROM == 1) && (i.POS > 10000) && (i.POS < 100200), vcf_table))

end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")

In [ ]:
# https://www.snpedia.com/index.php/Rs1042522
rs1042522 = ["chr17", 7676154]

# TP53
tp53 = ["chr17", 7661779:7687550]

# test set
t = ["chr1", 10000:12000]

## Get Genotype

In [ ]:
using Dates

start_time = now()

for value in vcf["chr17", 7661779]
    
    genotype = []
    
    allele_ref = value[:REF]
    
    allele_alt = value[:ALT]
    
    numeric_genotype = split(split(value[:GERM], ":")[1], "/")
    
    if occursin("|", numeric_genotype[1])
        
        numeric_genotype = split(numeric_genotype[1], "|")
        
    end
    
    counter = 1
    
        while counter < 3
        
            if "0" == numeric_genotype[counter]

                push!(genotype, allele_ref)

            elseif "1" == numeric_genotype[counter]

                push!(genotype, split(allele_alt, ",")[1])

            elseif "2" == numeric_genotype[counter]

                push!(genotype, split(allele_alt, ",")[2])
        
            end    
    
        counter += 1
        
        end

    println("Genotype: $genotype")
    
    println("\n")
    
end
  
end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")

## Tabix Regions

In [ ]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)